# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [138]:
import numpy as np
import requests as rq

# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Making random queries to the API

In [166]:
challenge_id = 'cool' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
query_results = query(challenge_id, queries)

print(query_results)

[ -1.   5.   9.  -5.  -8. -13.  32.   1. -12.   6.  34.  -5.  -8.  31.
  12.  11.  11.  11.   6.  19.  -1.   6.   6. -16.   9.  29. -11.  23.
  10.   5. -37.  21.  -5. -16. -25.   4.   0.   7.   7.  18.   2. -10.
 -19.  14.  -7.   2.  17.  -8.  -1.  13. -18.  10.  18.  -6.  22.  -6.
 -15.  19.   0. -17.  36.   4.  29. -40.  -3. -15. -10. -14.  21.  16.
   1. -12. -29.  10. -13.  13. -29.  12.  -2.  -9. -10.   4.  24.  74.
   9. -10.  18.  20.   5.  -5.  -5.  21.   8. -14.  -1.   4. -32.  -3.
   2.   1. -20.  -2. -22.  13. -23.  22.   2. -14. -18.  20.   7.  -6.
  -2. -15.  22.  31. -11.  -7. -11.  -6.   1. -13. -11.  12.  -6. -17.
  10.  11. -17.  21.  -5.  29.  -7. -16. -10.  12. -16. -18.   4. -17.
 -22.  20. -10.  13.  -6. -24.  27.  15.  -1.  19.  11. -22.  -6. -24.
 -23. -10.   1.   3.  20. -22.   9.  -9. -10.  28.  13.   3. -12. -12.
  13.   6. -11.  -1.   3.   5.  -3.  40.   8. -22.   1.   5. -17.   2.
 -12.   1.  -7.  20.  -9.  15.  21. -16.  -4.   1.  -3.  10.   3.  16.
  14. 

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```



In [25]:
from scipy.optimize import linprog

In [167]:
A = np.stack((queries, -queries), axis = 1)
A = A.reshape([400,100])



query_results

c = np.zeros(n)

alphas = np.repeat(30, num_queries)
bound = np.stack((alphas + query_results, alphas - query_results), axis=1).reshape(400)

result = linprog(c, A_ub = A, b_ub = bound, bounds = [(-1,1)])
result_array = result.x

res_rounded = np.where(result_array >= 0, 1, -1)

print(res_rounded)
best_query_result = query('coolID', res_rounded, submit=True)
print((1 + best_query_result / n) / 2)

[-1  1 -1  1 -1  1 -1  1  1 -1  1  1  1  1  1 -1  1 -1  1  1  1  1  1  1
  1 -1  1 -1  1  1  1  1 -1  1  1  1  1 -1  1  1 -1  1 -1 -1  1 -1  1  1
  1  1  1 -1  1  1  1  1 -1  1  1  1 -1  1 -1 -1  1 -1  1 -1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1 -1  1  1 -1 -1 -1  1  1  1 -1  1  1
  1  1 -1 -1]
0.56
